<a href="https://colab.research.google.com/github/U-Power/Coursera_Capstone/blob/main/TorontoNeighborhoods1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmenting and Clustering Neighborhoods in Toronto**

## Getting Toronto information on the web and scraping the url page

In [34]:
# import the beautiful soup package to scrape the web page
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [35]:
# create a bs4 object
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
soup.prettify();

In [36]:
# from a closer look you can find the lines containing the three informations needed for each PostalCode (TAG 'p')
soup.tbody.find_all('p');

In [37]:
# extract the row string containing the 3 metrics needed
row_data = [element.text for element in soup.tbody.find_all('p')]
row_data[0:5]

['M1ANot assigned\n',
 'M2ANot assigned\n',
 'M3ANorth York(Parkwoods)\n',
 'M4ANorth York(Victoria Village)\n',
 'M5ADowntown Toronto(Regent Park / Harbourfront)\n']

In [38]:
# extract the postal codes, getting the first 3 digits of each string. Add them to a list
pc = [];
for i in range(len(row_data)):
  pc.append(row_data[i][0:3])
pc[0:5]

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']

In [39]:
# extract the rest of data and then split it, adding the first element of the split to the Borough list
borough = [];
for i in range(len(row_data)):
  borough.append(row_data[i][3:].split('(')[0].split('\n')[0])
borough[0:5]


['Not assigned',
 'Not assigned',
 'North York',
 'North York',
 'Downtown Toronto']

In [40]:
# extract the rest of data and then split it, adding the second element of the split to the Neighborhoods list (add 'Not assigned', first element if there is not a second one)
neighb = [];
for i in range(len(row_data)):
  try: neighb.append(row_data[i][3:].split('(')[1].split(')')[0].strip().replace(' / ',', ')) 
  except: neighb.append(row_data[i][3:].split('(')[0].split('\n')[0])

neighb[0:5]

['Not assigned',
 'Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Regent Park, Harbourfront']

## Create a pandas DataFrame and cleanse the data

In [41]:
# create a dictionary to store the 3 columns needed in the dataFrame
data = {}
data['PostalCode'] = pc
data['Borough'] = borough
data['Neighborhood'] = neighb

#create a DataFrame
df = pd.DataFrame(data)
df[0:5]

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [42]:
# take a look at the data grouping by 'Borough' and adjust the rows labels
df.groupby('Borough').describe()

PostalCode  ... Neighborhood
                                                        count  ...         freq
Borough                                                        ...             
Central Toronto                                             9  ...            1
Downtown Toronto                                           17  ...            1
Downtown TorontoStn A PO Boxes25 The Esplanade              1  ...            1
East Toronto                                                4  ...            1
East TorontoBusiness reply mail Processing Cent...          1  ...            1
East York                                                   4  ...            1
East YorkEast Toronto                                       1  ...            1
Etobicoke                                                  11  ...            1
EtobicokeNorthwest                                          1  ...            1
MississaugaCanada Post Gateway Processing Centre            1  ...            1
North York                                                 24  ...            4
Not assigned                                               77  ...           77
Queen's Park                                                1  ...            1
Scarborough                                                17  ...            1
West Toronto                                                6  ...            1
York                                                        5  ...            1

[16 rows x 8 columns]

In [43]:
# adjust the label of some borough names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [44]:
print('The dimensions of the Data Frame are:', df.shape)

The dimensions of the Data Frame are: (180, 3)


In [45]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


### It is possible to add more rows for every Neighborhood (to make a better analysis)

In [46]:
# try to duplicate the rows which have multiple Neighborhood values in order to map them separately
new_df = pd.DataFrame(df.Neighborhood.str.split(', ').tolist(), index=df.PostalCode).stack()
new_df = new_df.reset_index([0, 'PostalCode'])
new_df.columns = ['PostalCode', 'Neighborhood']
new_df.head(10)

,PostalCode,Neighborhood
0,M1A,Not assigned
1,M2A,Not assigned
2,M3A,Parkwoods
3,M4A,Victoria Village
4,M5A,Regent Park
5,M5A,Harbourfront
6,M6A,Lawrence Manor
7,M6A,Lawrence Heights
8,M7A,Ontario Provincial Government
9,M8A,Not assigned


In [47]:
# create a separate df which links postal codes to boroughs
df_bor = df.drop('Neighborhood', axis=1)
df_bor

,PostalCode,Borough
0,M1A,Not assigned
1,M2A,Not assigned
2,M3A,North York
3,M4A,North York
4,M5A,Downtown Toronto
...,...,...
175,M5Z,Not assigned
176,M6Z,Not assigned
177,M7Z,Not assigned
178,M8Z,Etobicoke


In [48]:
# merge the dataFrames to have a complete single row for each neighborhood with the borough assigned
df_adj = new_df.merge(df_bor, how='inner')
df_adj

,PostalCode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,Parkwoods,North York
3,M4A,Victoria Village,North York
4,M5A,Regent Park,Downtown Toronto
...,...,...,...
288,M8Z,The Queensway West,Etobicoke
289,M8Z,South of Bloor,Etobicoke
290,M8Z,Kingsway Park South West,Etobicoke
291,M8Z,Royal York South West,Etobicoke


In [49]:
# ideally the dataframe can be cleansed removing the rows with 'Not assigned' Neighborhoods (to ease the coord fetch API in the next section)
df_adj = df_adj[df_adj.Neighborhood != 'Not assigned']
df_adj

,PostalCode,Neighborhood,Borough
2,M3A,Parkwoods,North York
3,M4A,Victoria Village,North York
4,M5A,Regent Park,Downtown Toronto
5,M5A,Harbourfront,Downtown Toronto
6,M6A,Lawrence Manor,North York
...,...,...,...
287,M8Z,Mimico NW,Etobicoke
288,M8Z,The Queensway West,Etobicoke
289,M8Z,South of Bloor,Etobicoke
290,M8Z,Kingsway Park South West,Etobicoke


## Use the geopy Python library to get Neighborhood coordinates
**In order to simplify you can drop this section and use the CSV provided instead.**
**It is suggested since getting the coordinates for every Neighborhood does not give additional informations compared with using postal codes. Also, it takes 2-3 min to run the cycle for all the Neighborhoods**

In [50]:
# import the libraries
!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [51]:
# get the coordinates for every neighborhood in Toronto and append them to the dataFrame
latitudes = [];
longitudes = [];
 
for i in range(len(df_adj['Neighborhood'])):
  try:
    address = 'Toronto, Ontario, ' + df_adj['Neighborhood'][i]
    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latitudes.append(latitude)
    longitudes.append(longitude)
  except:
    latitudes.append('NaN')
    longitudes.append('NaN')


df_fin = df_adj
df_fin['Latitude'] = latitudes
df_fin['Longitude'] = longitudes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [52]:
# pay attention at the unique coordinates values we get for every Neighborhood
# it can be compared with the unique values we get from the csv provided
df_fin.describe()

,PostalCode,Neighborhood,Borough,Latitude,Longitude
count,216,216,216,216,216
unique,103,208,15,147,147
top,M9W,Downsview,Etobicoke,NaN,NaN
freq,9,4,44,56,56


In [68]:
# the NaN rows can be eliminated in order to ease the next part of the 4square API 
df_fin = df_fin[df_fin.Latitude != 'NaN']
df_fin.describe()

,PostalCode,Neighborhood,Borough,Latitude,Longitude
count,160,160,160,160.000000,160.000000
unique,88,152,13,146.000000,146.000000
top,M5V,Downsview,Etobicoke,43.749299,-79.462248
freq,6,4,32,4.000000,4.000000


In [53]:
# get the csv file, put it in a dataFrame
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
latlong_df = pd.read_csv(url)
latlong_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
# notice that the unique coordinates values in the CSV and in the dataframe built with the API
print('The number of neighborhood coord for the CSV: {} \nThe number of neighborhood coord fetched with the API: {} \nSo we can make a better analysis using the data fetched from the API'.format(latlong_df['Latitude'].nunique(), df_fin['Latitude'].nunique()))

The number of neighborhood coord for the CSV: 103 
The number of neighborhood coord fetched with the API: 146 
So we can make a better analysis using the data fetched from the API


# Segmenting Toronto Neighborhoods using 4square API

## Map the neighborhood with Folium library

In [56]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries seccessfully imported')

Libraries seccessfully imported


In [57]:
# get the latitude and longitude of Toronto
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [70]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

# add markers to map
for lat, lng, borough, neighborhood in zip(df_fin['Latitude'], df_fin['Longitude'], df_fin['Borough'], df_fin['Neighborhood']):
  try:
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
  except Exception:
    pass

map_toronto

## Now let's get informaton on the venues nearby every Neighborhood

In [63]:
#@title Personal data

CLIENT_ID = '5JKJ2KFPVYPAREZ2AOHPCMO3WXDUSKYGR2ZAO0QO3JUQOQC0' # your Foursquare ID
CLIENT_SECRET = 'VUFHA0PCC343VCPF4ULZLUHVNRZMNJDPFNT0EWW3HVHVKDY2' # your Foursquare Secret
ACCESS_TOKEN = 'G51ZR5XS3NMIKFO53YZ4RKDVW3OO1GBG1HIZ0VEHXCE0ROND' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


### Run the 4square API to fetch infos on the venues of each Neighborhood

In [71]:
# let's make an example
neighborhood_latitude = df_fin.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_fin.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_fin.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park are 43.7587999, -79.3201966.


In [72]:
radius = 500
#search_query = 'TopPicks' # this query reduces the venues fetched by the API so it depends case by case if can be useful
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=5JKJ2KFPVYPAREZ2AOHPCMO3WXDUSKYGR2ZAO0QO3JUQOQC0&client_secret=VUFHA0PCC343VCPF4ULZLUHVNRZMNJDPFNT0EWW3HVHVKDY2&ll=43.7587999,-79.3201966&oauth_token=G51ZR5XS3NMIKFO53YZ4RKDVW3OO1GBG1HIZ0VEHXCE0ROND&v=20180605&radius=500&limit=100'

In [86]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '609cefe4499626724f486671'},
 'notifications': [{'item': {'unreadCount': 0}, 'type': 'notificationTray'}],
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b8991cbf964a520814232e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/caribbean_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d144941735',
         'name': 'Caribbean Restaurant',
         'pluralName': 'Caribbean Restaurants',
         'primary': True,
         'shortName': 'Caribbean'}],
       'id': '4b8991cbf964a520814232e3',
       'location': {'address': '81 Underhill drive',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 381,
        'formattedAddress': ['81 Underhill drive',
         'Toronto ON M3A 1Z5'

In [74]:
# function to create a dataframe with the API informations fetched
# credits for the function to Alex Aklson and Polong Lin
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronto_venues = getNearbyVenues(names=df_fin['Neighborhood'],
                                   latitudes=df_fin['Latitude'],
                                   longitudes=df_fin['Longitude']
                                  )

Regent Park
Harbourfront
Lawrence Manor
Lawrence Heights
Ontario Provincial Government
Islington Avenue
Don Mills
Parkview Hill
Woodbine Gardens
Ryerson
Glencairn
West Deane Park
Princess Gardens
Martin Grove
Islington
Port Union
Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate
Bloordale Gardens
Markland Wood
Guildwood
Morningside
West Hill
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Downsview North
Thorncliffe Park
Dovercourt Village
Henry Farm
Oriole
Northwood Park
Union Station
Toronto Islands
Little Portugal
Trinity
Kennedy Park
Ionview
East Birchmount Park
Bayview Village
Downsview
The Danforth West
Riverdale
Parkdale Village
Exhibition Place
Golden Mile
Clairlea
Oakridge
York Mills
Silver Hills
Downsview
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Humber Summit
Cliffside
Cliffcrest
Scarborough Village West
Willowdale
Newtonbrook
Downsview
Studio District
Bedford Park
Lawrence Manor 

In [80]:
# take a look at the resul set
print(toronto_venues.shape)
toronto_venues.head()

(4906, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Regent Park,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Regent Park,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
3,Regent Park,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
4,Regent Park,43.7588,-79.320197,TD Canada Trust,43.757440,-79.314838,Bank


### Analyse each Neighborhood

In [81]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood,39,39,39,39,39,39
Bathurst Quay,92,92,92,92,92,92
Bayview Village,100,100,100,100,100,100
Beaumond Heights,3,3,3,3,3,3
...,...,...,...,...,...,...
Woodbine Gardens,3,3,3,3,3,3
Woodbine Heights,27,27,27,27,27,27
York Mills,25,25,25,25,25,25


In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 310 uniques categories.


In [89]:
# create dummies for the venues categories in order to run a clustering model
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(4906, 310)


0    Regent Park
1    Regent Park
2    Regent Park
3    Regent Park
4    Regent Park
Name: Neighborhood, dtype: object

In [90]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,...,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.0,0.0,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00,0.00000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
1,Alderwood,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.0,0.0,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.076923,0.051282,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.025641,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00,0.00000,0.00000,0.0,0.025641,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025641,0.00000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
2,Bathurst Quay,0.0,0.0,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.00,0.01087,0.0,0.0,0.01087,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.021739,0.010870,0.01087,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.010870,0.0,...,0.0,0.0,0.0,0.0,0.0,0.01087,0.0,0.0,0.01087,0.0,0.0000,0.0,0.021739,0.0,0.0,0.00,0.01087,0.01087,0.0,0.021739,0.021739,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010870,0.01087,0.0,0.010870,0.0,0.0,0.00,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.0,0.0,0.01,0.00000,0.0,0.0,0.02000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.010000,0.000000,0.01000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.01000,0.0,0.0000,0.0,0.010000,0.0,0.0,0.01,0.00000,0.01000,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010000,0.00000,0.0,0.000000,0.0,0.0,0.01,0.0,0.0
4,Beaumond Heights,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.0,0.0,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00,0.00000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Woodbine Gardens,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.0,0.0,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00,0.00000,0.00000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000

In [91]:
# function to sort the venues in descending order
# credits for the function to Alex Aklson and Polong Lin
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
# new dataframe that includes the cluster as well as the top N venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,General Entertainment,College Stadium,Skating Rink,Women's Store,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
1,Alderwood,Coffee Shop,Café,Bakery,Pizza Place,Bank,Dance Studio,Pub,Frozen Yogurt Shop,Bookstore,Dessert Shop
2,Bathurst Quay,Coffee Shop,Clothing Store,Hotel,Café,Restaurant,Seafood Restaurant,Japanese Restaurant,Salad Place,Bakery,Sushi Restaurant
3,Bayview Village,Coffee Shop,Café,Hotel,Gym,Restaurant,Italian Restaurant,Japanese Restaurant,Clothing Store,American Restaurant,Gastropub
4,Beaumond Heights,Gym,Intersection,Electronics Store,Women's Store,Doctor's Office,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant


## Run k-means to cluster the neighborhood into N clusters

In [94]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [157]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 0, 0, 3, 3, 3, 3, 3, 0, 0],
      dtype=int32)

In [158]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_fin

# merge toronto_grouped with df_fin to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

ValueError: ignored

In [148]:
# look for NaN values in the dataframe
print('Are there NaN values in the dataframe?:', toronto_merged.isnull().values.any(), '\n')
is_NaN = toronto_merged.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = toronto_merged[row_has_NaN]

print(rows_with_NaN)

Are there NaN values in the dataframe?: True 

   PostalCode Neighborhood  ... 9th Most Common Venue 10th Most Common Venue
28        M1C   Port Union  ...                   NaN                    NaN

[1 rows x 16 columns]


In [155]:
# eliminate the useless row
toronto_merged.drop([rows_with_NaN.index[0]], inplace = True)

KeyError: ignored

In [156]:
# check if the cleansing is ok
print('Are there NaN values in the dataframe?:', toronto_merged.isnull().values.any(), '\n')

Are there NaN values in the dataframe?: False 



In [159]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(round(cluster-1))],
        fill=True,
        fill_color=rainbow[int(round(cluster-1))],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Look at the clusters

In [166]:
# cluster 1 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0].head()


,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M7A,Ontario Provincial Government,Queen's Park,43.7221,-79.4375,0.0,Doctor's Office,Bank,Park,Electronics Store,Kids Store,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
16,M4B,Woodbine Gardens,East York,43.8049,-79.1658,0.0,Caribbean Restaurant,Park,Fast Food Restaurant,Ethiopian Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
22,M9B,West Deane Park,Etobicoke,43.7121,-79.3026,0.0,Coffee Shop,Home Service,Park,Bakery,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant
41,M1E,Guildwood,Scarborough,43.6999,-79.3193,0.0,Skating Rink,Pharmacy,Park,Bus Stop,Athletics & Sports,Dance Studio,Intersection,Women's Store,Electronics Store,Doner Restaurant
52,M1G,Woburn,Scarborough,43.6353,-79.5637,0.0,Intersection,Park,Convenience Store,Donut Shop,Sandwich Place,Coffee Shop,Print Shop,Women's Store,Doner Restaurant,Dumpling Restaurant


In [161]:
# cluster 2 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1].head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
271,M8Y,Old Mill South,Etobicoke,43.8162,-79.3145,1.0,Playground,Women's Store,Ethiopian Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


In [162]:
# cluster 3 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2].head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M6B,Glencairn,North York,43.7063,-79.3219,2.0,Construction & Landscaping,Women's Store,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
166,M5R,North Midtown,Central Toronto,43.7374,-79.4109,2.0,Construction & Landscaping,Women's Store,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


In [163]:
# cluster 4 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3].head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Regent Park,Downtown Toronto,43.7588,-79.3202,3.0,Caribbean Restaurant,Discount Store,Electronics Store,Coffee Shop,Pizza Place,Chinese Restaurant,Bus Line,Liquor Store,Pharmacy,Bank
5,M5A,Harbourfront,Downtown Toronto,43.7327,-79.3112,3.0,Thai Restaurant,Spa,Asian Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Filipino Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
6,M6A,Lawrence Manor,North York,43.6607,-79.3605,3.0,Coffee Shop,Restaurant,Thai Restaurant,Pool,Food Truck,Sushi Restaurant,Fast Food Restaurant,Beer Store,Auto Dealership,Grocery Store
7,M6A,Lawrence Heights,North York,43.6401,-79.3801,3.0,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Plaza,Chinese Restaurant,Sushi Restaurant,Bank
10,M9A,Islington Avenue,Etobicoke,43.7228,-79.4509,3.0,Clothing Store,Coffee Shop,Restaurant,Furniture / Home Store,Women's Store,Sporting Goods Shop,American Restaurant,Men's Store,Toy / Game Store,Jewelry Store


In [164]:
# cluster 5 
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4].head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M9C,Eringate,Etobicoke,43.7755,-79.135,4.0,Park,Women's Store,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
48,M6E,Caledonia-Fairbanks,York,43.6623,-79.5765,4.0,Park,Electronics Store,Women's Store,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Ethiopian Restaurant
107,M1L,Oakridge,Scarborough,43.7541,-79.5045,4.0,Park,Baseball Field,Women's Store,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
226,M9V,Humbergate,Etobicoke,43.6995,-79.5563,4.0,Park,Women's Store,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant
